# Logistic classification

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data and preprocessing
data = load_breast_cancer()
X = data.data
y = data.target.reshape(-1, 1)  # Convert to column vectors

# Divide the training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Data standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)     # Calculate the mean, standard deviation, and convert
X_test = scaler.transform(X_test)       # Use the mean and standard deviation calculated above

# Convert to tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Define model
class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return self.linear(x)

# Init model
input_dim = X_train.shape[1]
model = LogisticRegression(input_dim)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)  # alpha is 0.1

# Train model
epochs = 200
for epoch in range(epochs):
    # Forward propagation
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # Backward propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print loss
    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Evaluate model
with torch.no_grad():       # No gradient calculation
    y_pred = model(X_test)
    y_prob = torch.sigmoid(y_pred)  # Sigmoid 
    y_class = (y_prob > 0.5).float() 
    
    # Calculate accuracy
    accuracy = (y_class == y_test).float().mean()
    print(f"\nTest Accuracy: {accuracy.item()*100:.2f}%")

Epoch [20/200], Loss: 0.0699
Epoch [40/200], Loss: 0.0623
Epoch [60/200], Loss: 0.0580
Epoch [80/200], Loss: 0.0545
Epoch [100/200], Loss: 0.0519
Epoch [120/200], Loss: 0.0501
Epoch [140/200], Loss: 0.0488
Epoch [160/200], Loss: 0.0476
Epoch [180/200], Loss: 0.0465
Epoch [200/200], Loss: 0.0455

Test Accuracy: 98.25%
